In [1]:
# Install dependencies (uncomment the following lines by removing ''' if you haven't installed the dependencies yet)
'''
%pip install finvizfinance
%pip install pandas
%pip install transformers
%pip install yfinance
%pip install goose3
%pip install requests
%pip install ipywidgets

%pip install torch
%pip install tensorflow
%pip install nltk
import nltk
nltk.download('punkt')
'''


"\n%pip install finvizfinance\n%pip install pandas\n%pip install transformers\n%pip install yfinance\n%pip install goose3\n%pip install requests\n%pip install ipywidgets\n\n%pip install torch\n%pip install tensorflow\n%pip install nltk\nimport nltk\nnltk.download('punkt')\n"

In [19]:
# Import libraries
from finvizfinance.screener.overview import Overview # type: ignore
from finvizfinance.quote import finvizfinance        # type: ignore
from IPython.display import display                  # type: ignore

import pandas as pd                 # type: ignore
from transformers import pipeline   # type: ignore
import yfinance as yf               # type: ignore
from goose3 import Goose            # type: ignore
from requests import get            # type: ignore

from nltk.tokenize import sent_tokenize # type: ignore
from transformers import AutoTokenizer  # type: ignore

import csv
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Other general settings
pd.set_option('display.max_colwidth', None) # Display full text in pandas dataframe / no line wrapping


In [ ]:
# Create filters dictionary, i.e filter the stocks based on the following criteria:
FILTERS_DICT = {
    'Performance 2': 'Today +10%',     # Day increase 10%
    'Relative Volume': 'Over 5',       # High Relative Volume
    'Price': 'Under $20',              # Price under 20 USD
    'Shares Outstanding': 'Under 10M'  # Float under 10 million
}

# Alternative filtering to consider:
'''
FILTERS_DICT = {'Debt/Equity':'Under 1',                 # Positive Operating Margin
                'PEG':'Low (<1)',                        # Debt-to-Equity ratio under 1
                'Operating Margin':'Positive (>0%)',     # Low P/B (under 1)
                'P/B':'Low (<1)',                        # Low P/E ratio (under 15)
                'P/E':'Low (<15)',                       # Low PEG ratio (under 1)
                'InsiderTransactions':'Positive (>0%)<'} # Positive Insider Transactions
'''


# The filters and general manual link for the finvizfinance library: https://finvizfinance.readthedocs.io/_/downloads/en/latest/pdf/ 
# Possible filters can be found by running the following code:
'''
from finvizfinance.screener.overview import Overview
foverview2 = Overview()    # Create Overview object
foverview2.get_filters()   # Get list of all possible filters
'''
# And after to see the possible options for a filter, run the following code:
'''
foverview2.get_filter_options('Relative Volume') # Get list of all possible options for a filter, example on 'Relative Volume'
'''

In [23]:
# Function to get the filtered stocks:
def get_filtered_stocks():
    """
    Returns a list of tickers with:

    """
    
    foverview = Overview()
    foverview.set_filter(filters_dict=FILTERS_DICT)
    df_overview = foverview.screener_view()
    if not os.path.exists('out'): #ensures you have an 'out' folder ready
        os.makedirs('out')
    df_overview.to_csv('out/Overview.csv', index=False)
    
    tickers = df_overview['Ticker'].to_list()
    display(df_overview)
    return tickers



undervalued = get_filtered_stocks()


,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
0,CMND,Clearmind Medicine Inc,Healthcare,Biotechnology,Canada,4370000.0,NaN,1.38,0.1795,47782864.0
1,MCVT,Mill City Ventures III Ltd,Financial,Credit Services,USA,19170000.0,NaN,3.00,0.1111,14536.0
2,MSGM,Motorsport Games Inc,Communication Services,Electronic Gaming & Multimedia,USA,8680000.0,NaN,3.19,0.4055,5546681.0
3,NUWE,Nuwellis Inc,Healthcare,Medical Devices,USA,1680000.0,NaN,0.25,0.4474,127464529.0
4,NUZE,Nuzee Inc,Consumer Defensive,Packaged Foods,USA,2710000.0,NaN,2.12,0.5905,25280575.0
5,NYC,American Strategic Investment Co,Real Estate,Real Estate Services,USA,20370000.0,NaN,8.15,0.4052,272847.0
6,OGEN,Oragenics Inc,Healthcare,Biotechnology,USA,5990000.0,NaN,1.34,0.2383,529530.0
7,PLTN,Plutonian Acquisition Corp,Financial,Shell Companies,USA,53200000.0,99.25,10.64,0.1141,751073.0
8,RNAZ,TransCode Therapeutics Inc,Healthcare,Biotechnology,USA,7000000.0,NaN,1.07,0.3072,2878359.0
9,VINO,Gaucho Group Holdings Inc,Real Estate,Real Estate - Diversified,USA,4210000.0,NaN,5.26,0.1288,3704711.0


In [17]:
# Function to get the sentiment of the news articles for a given ticker.
# This may run for a good few minutes, depending on the number of filtered tickers / articles. (seen approx 2-7 minutes total)


ALLOW_TOKENIZATION = False # True: the model will feed the article into the model in chunks of 512 tokens, 
#                            False: the model will consider only the first sentences of the article until the total number of tokens does not exceed 512

def get_ticker_news_sentiment(ticker):
    """
    Returns a Pandas dataframe of the given ticker's most recent news article headlines,
    with the overal sentiment of each article.

    Args:
        ticker (string)

    Returns:
        pd.DataFrame: {'Date', 'Article title', Article sentiment'}
    """
    
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

    ticker_news = yf.Ticker(ticker)
    news_list = ticker_news.get_news()
    extractor = Goose()
    pipe = pipeline("text-classification", model="ProsusAI/finbert")
    data = []
    for dic in news_list:
        title = dic['title']
        response = get(dic['link'])
        article = extractor.extract(raw_html=response.content)
        text = article.cleaned_text
        date = article.publish_date
        if len(text) > 512:
            if ALLOW_TOKENIZATION: # feed the article into the model in chunks of 512 tokens
                inputs = tokenizer.encode_plus(
                    text,
                    max_length=510,
                    truncation='longest_first',  # Truncate the longest sequences first
                    padding='max_length',  # Pad sequences to the max length
                    return_tensors='pt',  # Return PyTorch tensors
                )
                
                # Convert tensor to list and then to string
                input_ids = inputs["input_ids"].tolist()[0]
                new_text = tokenizer.decode(input_ids)
            
            else: # count the sentences until the total number of tokens does not exceed 512 (consider only first sentences of the article)
                # Split the text into sentences
                sentences = sent_tokenize(text)

                # Initialize an empty string for the new text
                new_text = ''

                # Add sentences to the new text until it exceeds 512 tokens
                for sentence in sentences:
                    if len(new_text.split()) + len(sentence) > 512:
                        new_text += ' ' + sentence
                    break

            # Now you can pass 'inputs' to your model
            results = pipe(new_text)

            data.append({'Ticker':f'{ticker}',
                         'Date':f'{date}',
                         'Article title':f'{title}',
                         'Article sentiment':results[0]['label']})

        else:
            results = pipe(text)
            data.append({'Ticker':f'{ticker}',
                         'Date':f'{date}',
                         'Article title':f'{title}',
                         'Article sentiment':results[0]['label']})
    df = pd.DataFrame(data)
    return df

def generate_csv(ticker):
    get_ticker_news_sentiment(ticker).to_csv(f'out/{ticker}.csv', index=False)



#undervalued = get_filtered_stocks()

sentiments = []
for ticker in undervalued:
    generate_csv(ticker)
    sentiments.append(get_ticker_news_sentiment(ticker))



In [33]:
# Print the news:
for i in range(len(sentiments)):
    display(sentiments[i])  # This will print the first element of each inner list


,Ticker,Date,Article title,Article sentiment
0,CETX,2024-05-03T13:50:00.000Z,"Cemtrex, Inc. Announces Closing of $10 Million Upsized Underwritten Public Offering",neutral
1,CETX,2024-05-01T18:29:44.000Z,Top Midday Decliners,neutral
2,CETX,2024-05-01T12:59:00.000Z,"Cemtrex, Inc. Announces Pricing of $10 Million Upsized Underwritten Public Offering",neutral
3,CETX,None,Top Premarket Decliners,positive
4,CETX,2024-04-30T18:42:43.000Z,Cemtrex Shares Drop After Filing Proposed Public Offering of $9 Million Units,neutral
5,CETX,None,Cemtrex’s Vicon Industries to Attend ISC West 2024,positive
6,CETX,None,Cemtrex’s Advanced Industrial Services Secures New Orders Totaling $2.1 Million,positive
7,CETX,None,Cemtrex Announces Beta Release of Pioneering AI Gun Detection Feature,positive


,Ticker,Date,Article title,Article sentiment
0,CMND,2024-05-07T11:46:00.000Z,Clearmind Medicine Secures Exclusive Global Rights to Breakthrough Psychedelic Compounds for PTSD Treatment,neutral
1,CMND,2024-04-30T20:15:00.000Z,Clearmind Applies to Cease Being a Reporting Issuer in Canada,neutral
2,CMND,2024-04-17T11:49:00.000Z,Clearmind Medicine Announces Exclusive Licensing Agreement for Generation 3.0 Psychedelic Compounds for the Treatment of Mental Disorders,neutral
3,CMND,2024-04-10T11:17:00.000Z,Clearmind Medicine Files U.S. Patent Application for Psychedelic-Based Treatment for Eating Disorders,neutral
4,CMND,None,Clearmind Medicine CEO Issues Letter to Shareholders,positive
5,CMND,None,Clearmind Medicine Announces International Patent Application for Preventing and Treating Depression,positive
6,CMND,None,Breakthrough in Wellness: Clearmind Medicine's Psychedelic Treatment Granted Divisional Patent Approval in China,positive
7,CMND,None,Clearmind Medicine Obtains Clearance for its Psychedelic- Based Alcoholism Clinical Trial,positive


""


,Ticker,Date,Article title,Article sentiment
0,MSGM,2024-05-02T13:00:00.000Z,Motorsport Games to Report First Quarter 2024 Financial Results,neutral
1,MSGM,2024-05-01T12:30:00.000Z,Motorsport Games Announces Sale of Traxion,neutral
2,MSGM,2024-04-18T13:00:00.000Z,Motorsport Games and British Touring Car Championship Announce Settlement and New Licensing Agreement,neutral
3,MSGM,None,Motorsport Games Inc. (NASDAQ:MSGM) Q4 2023 Earnings Call Transcript,positive
4,MSGM,None,Motorsport Games Reports Fourth Quarter & Full Year 2023 Financial Results,positive
5,MSGM,None,Motorsport Games to Report Fourth Quarter of 2023 & Full Year 2023 Financial Results,positive
6,MSGM,None,"Official 24 Hours of Le Mans Game, Le Mans Ultimate, available today",positive


,Ticker,Date,Article title,Article sentiment
0,NUWE,2024-05-07T12:00:00.000Z,"Nuwellis, Inc. Announces First Quarter 2024 Financial Results",neutral
1,NUWE,2024-05-02T13:00:00.000Z,Nuwellis Receives USPTO Notice of Patent Allowance for Innovative Pediatric CRRT Technology,neutral
2,NUWE,None,Nuwellis Announces Closing of $2.7 Million Public Offering,positive
3,NUWE,2024-04-30T20:41:00.000Z,"Nuwellis, Inc. To Report First Quarter 2024 Financial Results on May 7, 2024",neutral
4,NUWE,2024-04-26T13:17:00.000Z,Nuwellis Announces Pricing of $2.7 Million Public Offering,neutral
5,NUWE,2024-04-09T13:00:00.000Z,Nuwellis Announces Launch of Ultrafiltration Therapy Using Aquadex Smartflow® System at Henry Ford Health,neutral
6,NUWE,None,Nuwellis Full Year 2023 Earnings: EPS Beats Expectations,positive
7,NUWE,None,Late-Breaking Data Highlighting Benefits of Ultrafiltration Therapy Using the Aquadex System for Heart Failure Patients was Presented at the Technology and Heart Failure Therapeutics Conference,positive


...

,Ticker,Date,Article title,Article sentiment
0,NYC,2024-05-07T10:00:00.000Z,"BELLEVUE CAPITAL PARTNERS, LLC LAUNCHES TENDER OFFER FOR SHARES OF AMERICAN STRATEGIC INVESTMENT CORP.",neutral
1,NYC,None,"American Strategic Investment Co. Announces New Leasing, 87.2% Occupancy",positive
2,NYC,2024-04-30T10:00:00.000Z,American Strategic Investment Co. to Accelerate Evolution of Business Model,neutral
3,NYC,2024-04-25T10:00:00.000Z,American Strategic Investment Co. Announces Release Date for First Quarter Results,neutral
4,NYC,2024-04-01T20:15:00.000Z,American Strategic Investment Co. Announces Fourth Quarter 2023 Results,neutral
5,NYC,2024-03-11T10:00:00.000Z,American Strategic Investment Co. Announces Release Date for Fourth Quarter and Full Year 2023 Results,neutral


,Ticker,Date,Article title,Article sentiment
0,OGEN,2024-05-07T12:00:00.000Z,Oragenics Partners with Avance Clinical for Phase II Concussion Trial in Australia,neutral
1,OGEN,2024-04-19T21:50:00.000Z,"Oragenics, Inc. Announces Notification of Noncompliance with Additional NYSE American Continued Listing Standards",neutral
2,OGEN,2024-04-01T13:05:00.000Z,"Oragenics, Inc. Files 10K and Provides Company Update",neutral
3,OGEN,2024-03-18T11:00:00.000Z,"Oragenics Appoints James Kelly MD, Chief Medical Officer, to Lead Phase II Clinical Trials for Treating Concussion",neutral
4,OGEN,None,15 Highest Quality Probiotics For Gut Health,positive
5,OGEN,2024-03-05T12:45:00.000Z,"Oragenics, Inc. Prepares Drug for Phase II Clinical Trials to Treat Concussion",neutral
6,OGEN,2024-03-01T21:04:00.000Z,Oragenics Announces Closing of Public Offering,neutral
7,OGEN,2024-02-28T02:25:00.000Z,Oragenics Announces Pricing of Public Offering,neutral


,Ticker,Date,Article title,Article sentiment
0,RNAZ,2024-04-15T13:00:00.000Z,TransCode Therapeutics Announces FDA Clearance To Initiate Phase 1/2 Clinical Trial with TTX-MC138 in Patients with Advanced Solid Tumors,neutral
1,RNAZ,None,TransCode Therapeutics Reports 2023 Results; Provides Business Update,positive
2,RNAZ,None,"TransCode Therapeutics Announces Appointment of Daniel Vlock, M.D., as Chief Medical Officer",positive
3,RNAZ,None,TransCode Therapeutics and Akribion Genomics Report Progress Developing CRISPR-Derived Technology for Cancer Treatment,positive
4,RNAZ,None,TransCode Therapeutics To Present At 2024 RNA Leaders Europe Congress,positive
5,RNAZ,None,TransCode Therapeutics Reports Publication of United States Patent Application Covering TransCode’s RIG-I Agonist Immunotherapeutic,positive


,Ticker,Date,Article title,Article sentiment
0,SOPA,2024-05-01T13:00:00.000Z,Society Pass Set to Unlock and Monetize Current and Future Subsidiary Spin-Off Opportunities Through Strong and Flexible Financing,neutral
1,SOPA,2024-04-26T20:50:00.000Z,Society Pass Incorporated Announces 1-for-15 Reverse Split,neutral
2,SOPA,2024-04-24T13:53:00.000Z,"Society Pass Inc. Issues First Advance Notice to Strattners Bank, to Continue its Growth Plans",neutral


,Ticker,Date,Article title,Article sentiment
0,VINO,2024-05-07T15:15:00.000Z,"Gaucho Group Holdings, Inc. Supports Strong Investment in Argentina Following Endorsements from Global Leaders",positive
1,VINO,2024-05-06T13:15:00.000Z,"Gaucho Group Holdings, Inc. Observes Notable Surge In Buenos Aires Real Estate Market",neutral
2,VINO,2024-05-01T13:15:00.000Z,Gaucho Group Holdings Announces Completion of Reverse Stock Split,neutral
3,VINO,2024-04-29T13:00:00.000Z,"Gaucho Group Holdings, Inc. Announces Reverse Stock Split",neutral
4,VINO,2024-04-24T12:30:00.000Z,Gaucho Group Holdings Receives Notice from Nasdaq Regarding Continued Listing Requirements,neutral
5,VINO,2024-04-19T14:00:00.000Z,Gaucho Group Holdings Responds to Positive Economic Shifts in Argentina,neutral
6,VINO,2024-04-17T00:00:00.000Z,Gaucho Group Holdings Announces Delay in Annual Report Filing,neutral
7,VINO,None,Gaucho Group Holdings Announces Strategic Measures to Streamline Operations and Cost Efficiencies,positive


In [6]:
# Random junk code:
quote = finvizfinance('SGE')

df = quote.ticker_inside_trader()
from datetime import datetime
# Get today's date
today = datetime.today().date()

# Get the news and filter it to get only today's news
df = quote.ticker_news()
df = df[df['Date'].dt.date == today]
df


df = quote.ticker_fundament()
df

{'Company': 'Strong Global Entertainment Inc',
 'Sector': 'Technology',
 'Industry': 'Computer Hardware',
 'Country': 'USA',
 'Exchange': 'AMEX',
 'Index': '-',
 'P/E': '37.95',
 'EPS (ttm)': '0.05',
 'Insider Own': '78.01%',
 'Shs Outstand': '7.88M',
 'Perf Week': '61.54%',
 'Market Cap': '14.89M',
 'Forward P/E': '-',
 'EPS next Y': '-',
 'Insider Trans': '0.02%',
 'Shs Float': '1.73M',
 'Perf Month': '43.18%',
 'Income': '0.35M',
 'PEG': '-',
 'EPS next Q': '-',
 'Inst Own': '0.17%',
 'Short Float': '0.02%',
 'Perf Quarter': '68.75%',
 'Sales': '49.00M',
 'P/S': '0.30',
 'EPS this Y': '-',
 'Inst Trans': '-',
 'Short Ratio': '0.00',
 'Perf Half Y': '2.16%',
 'Book/sh': '0.99',
 'P/B': '1.91',
 'EPS next Y Percentage': '-',
 'ROA': '12.53%',
 'Short Interest': '0.00M',
 'Perf Year': '-',
 'Cash/sh': '0.69',
 'P/C': '2.72',
 'EPS next 5Y': '-',
 'ROE': '38.64%',
 '52W Range From': '1.02',
 '52W Range To': '4.35',
 'Perf YTD': '18.23%',
 'Dividend Est.': '-',
 'P/FCF': '11.37',
 'EPS p